In [16]:
import ee
import geemap
import os
import csv

In [17]:
Map =geemap.Map()
Map


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [18]:
# imput edited mangrove 2018 field data
fieldData = r'C:\Users\sgao5\Downloads\OutputData\OutputData\MangroveHainan18Fielddata.shp'
# fieldData = geemap.shp_to_ee(fieldData)
# Map.addLayer(fieldData,{},'Mangrove Field data 2018')

# add mangrove core area
# corearea = r'C:\Users\sgao5\Downloads\OutputData\OutputData\CoreArea.shp'
# corearea = geemap.shp_to_ee(corearea)
# add Hainan province rectangle 111.087E, 18.122N. 108.455E, 20.076N
Hainan = ee.Geometry.Rectangle([111.087, 18.122, 108.455, 20.15])

# to mask clouds based on the pixel_qa band of Landsat 8 SR data 
def maskL8sr(image):
   #Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = (1 << 3)
  cloudsBitMask = (1 << 5)
  # Get the pixel QA band.
  qa = image.select('pixel_qa')
  # Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                 .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
  return image.updateMask(mask)

# define image as landsat 8 SR dataset from May to August # Why use SR?
image = (ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(Hainan) \
    .filterDate('2018-05-01', '2018-8-31') \
    .map(maskL8sr)
    .median()) # 加括号
    #.filterMetadata('CLOUD_COVER_LAND', 'less_than', 50)
#composite = ee.Algorithms.Landsat.simpleComposite({
    #'collection': image,
    #'asFloat':True
#})
vispm = {
    'bands':['B5','B4','B3'],
    'min':0,
    'max':3000,
    'gamma':1.4
}
Map.addLayer(image, vispm,'Limage')

# set bands for prediction
bands = ['B1','B2','B3','B4','B5','B6','B7'] # why bring B1 to this list?


In [19]:
# define training points
mangrove_t = r'C:\Users\sgao5\Downloads\OutputData\18TVmangrive\mangrove18T.shp'
mangrove_t_shp = geemap.shp_to_ee(mangrove_t)
water_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\water18T.shp'
water_t_shp = geemap.shp_to_ee(water_t)
forest_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\forest18T.shp'
forest_t_shp = geemap.shp_to_ee(forest_t)
constructionLand_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\construct18T.shp'
constructionLand_t_shp = geemap.shp_to_ee(constructionLand_t)
cultivatedLand_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\culti18T.shp'
cultivatedLand_t_shp = geemap.shp_to_ee(cultivatedLand_t)
pond_t = r'C:\Users\sgao5\Downloads\Training18Data\Training18Data\pond18T.shp'
pond_t_shp = geemap.shp_to_ee(pond_t)
# define validating points
mangrove_v = r"C:\Users\sgao5\Downloads\OutputData\18TVmangrive\mangrove18V.shp"
mangrove_v_shp = geemap.shp_to_ee(mangrove_v)
water_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\water18V.shp"
water_v_shp = geemap.shp_to_ee(water_v)
forest_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\forest18V.shp"
forest_v_shp = geemap.shp_to_ee(forest_v)
constructionLand_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\construct18V.shp"
constructionLand_v_shp = geemap.shp_to_ee(constructionLand_v)
cultivatedLand_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\culti18V.shp"
cultivatedLand_v_shp = geemap.shp_to_ee(cultivatedLand_v)
pond_v = r"C:\Users\sgao5\Downloads\Validation18Data\Validation18Data\pond18V.shp"
pond_v_shp = geemap.shp_to_ee(pond_v)
# merge points
mgt_points = mangrove_t_shp.merge(water_t_shp).merge(forest_t_shp).merge(cultivatedLand_t_shp).merge(pond_t_shp).merge(constructionLand_t_shp)
mgv_points = mangrove_v_shp.merge(water_v_shp).merge(forest_v_shp).merge(cultivatedLand_v_shp).merge(pond_v_shp).merge(constructionLand_v_shp)
# mgt_points = mangrove_t_shp + water_t_shp + forest_t_shp + cultivatedLand_t_shp + pond_t_shp + constructionLand_t_shp
# mgv_points = mangrove_v_shp + water_v_shp + forest_v_shp + cultivatedLand_v_shp + pond_v_shp + constructionLand_v_shp
# mgt_points = [ee.Feature(mangrove_t_shp, {'landcover':0}),ee.Feature(water_t_shp, {'landcover':1}), ee.Feature(forest_t_shp, {'landcover':2}), ee.Feature(cultivatedLand_t_shp,{'landcover':3}), ee.Feature(pond_t_shp,{'landcover':4}), ee.Feature(constructionLand_t_shp,{'landcover':5})]
# mgt_points = ee.FeatureCollection(mgt_points)
# mgv_points = [ee.Feature(mangrove_v_shp, {'landcover':0}),ee.Feature(water_v_shp, {'landcover':1}), ee.Feature(forest_v_shp, {'landcover':2}), ee.Feature(cultivatedLand_v_shp,{'landcover':3}), ee.Feature(pond_v_shp,{'landcover':4}), ee.Feature(constructionLand_v_shp,{'landcover':5})]
# mgv_points = ee.FeatureCollection(mgv_points)

# Map.addLayer(mgpoints, {}, 'training points', False)

In [20]:
print(mgt_points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [110.60401000923413, 19.925015701055482]}, 'id': '1_1_1_1_1_0', 'properties': {'CID': 0, 'Landcover': 1}}


In [21]:
# landcover label
label = 'Landcover'
# overlay the points on the imagery to get training
training = image.select(bands).sampleRegions(**{
    'collection' : mgt_points,
    'properties' : [label],
    'scale' : 30,
})
        # deleted .select(bands)
validating = image.select(bands).sampleRegions(**{
    'collection' : mgv_points,
    'properties' : [label],
    'scale' : 30,
})
# Adds a column of deterministic pseudorandom numbers/ split training and varification dataset
trainingSample = training.randomColumn()
validatingSample = validating.randomColumn()

training = trainingSample.filter(ee.Filter.lt('random', 1))
validation = validatingSample.filter(ee.Filter.lt('random', 1))

In [22]:
# test points

print(trainingSample.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '1_1_1_1_1_0_0', 'properties': {'B1': 61.5, 'B2': 113, 'B3': 407.5, 'B4': 187, 'B5': 3791, 'B6': 1082.5, 'B7': 344, 'Landcover': 1, 'random': 0.7150004977851546}}


In [23]:
# Train the classifier
classifier = ee.Classifier.smileRandomForest(10).train(training, label, bands)

# classify the image with the same bands used for training
result = image.select(bands).classify(classifier)
# display the clusters with given colors
vis_params = {
    'min' : 1,
    'max' : 6,
    'palette' :['98ff00','62ffde','0b4a8b','fffff0','0dd66b','ff78d2']}
Map.addLayer(result, vis_params, 'classified')
Map

Map(bottom=754.0, center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

In [24]:
# add map elements
print('layer opacity:')
clusterLayer = Map.layers[-1]
clusterLayer.interact(opacity = (0,1,0.1))
# add legend
legend_dict = {
    '0 mangrove' : '98ff00',
    '1 water' : '62ffde',
    '2 forest' : '0b4a8b',
    '3 cultivated land' : 'fffff0',
    '4 pond' : '0dd66b',
    '5 construction land' : 'ff78d2'
}
Map.add_legend(legend_title = 'Hainan Mangrove Classification', legend_dict = legend_dict)
Map

layer opacity:


Map(bottom=58830.0, center=[18.890695349102117, -250.39489746093753], controls=(WidgetControl(options=['positi…

In [25]:
# Accuracy assessment
trainingAccuracy = classifier.confusionMatrix()
# validation dataset
validated = validation.classify(classifier)
testAccuracy = validated.errorMatrix('landcover', 'classification')

In [26]:
trainingAccuracy.accuracy().getInfo()

0.9756756756756757

In [27]:
trainingAccuracy.kappa().getInfo()

0.9661985609329147

In [ ]:
# Download confusion matrix
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')
with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(trainingAccuracy.getInfo())
with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(testAccuracy.getInfo())
# reclassify landcover map ?

# export the raster
# out_file = os.path.join(out_dir, 'mangroveClassification.tif')
# geemap.ee_export_image(landcover, filename=out_file, scale=900)